In [29]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

API_KEY = 'M3iWLItVErrMW5qgPwe5tzZFxjlROt1x'

with MPRester(API_KEY) as mpr:
    docs = mpr.materials.summary.search(
        energy_above_hull=(0, 0.008), band_gap=[0.85,2.15], is_stable=True, fields=["material_id"]
    )
    materials = [doc.material_id for doc in docs]

Retrieving SummaryDoc documents:   0%|          | 0/5081 [00:00<?, ?it/s]

In [30]:
def get_material_entries(material_ids, batch_size=1000):
    """Fetch material entries in batches."""
    mp_entries = []
    with MPRester(API_KEY) as mpr:
        for i in range(0, len(material_ids), batch_size):
            batch_ids = material_ids[i:i + batch_size]
            material_entries = mpr.materials.search(material_ids=batch_ids)
            mp_entries.extend(material_entries)
    return mp_entries

mp_entries = get_material_entries(materials)

Retrieving MaterialsDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/81 [00:00<?, ?it/s]

In [64]:
chemsys_list = [entry.chemsys for entry in mp_entries[:20] if hasattr(entry, 'chemsys')]
composition_list = [entry.composition for entry in mp_entries[:20] if hasattr(entry, 'composition')]
materials_gga = [material + "-GGA" for material in materials[:20]]

In [ ]:
pbx_entries = []
#pbx_data = []
#pbx_plots = []

with MPRester(API_KEY) as mpr:
    for i in chemsys_list:
        pourbaix_entries = mpr.get_pourbaix_entries(i)
        pbx_entries.append(pourbaix_entries)
        #pbx_diagram = PourbaixDiagram(pourbaix_entries)
        #pbx_data.append(pbx_diagram)  
        #pbx_plotter = PourbaixPlotter(pbx_diagram)
        #pbx_plots.append(pbx_plotter)

In [80]:
pbx_data = []
pbx_plots = []

for i in pbx_entries:
    pbx_diagram = PourbaixDiagram(i)
    pbx_data.append(pbx_diagram)
    pbx_plotter = PourbaixPlotter(pbx_diagram)
    pbx_plots.append(pbx_plotter)

In [73]:
matches = []

for comp in composition_list:
    comp_formula = comp.formula
    found_match = False  # Flag to track if a match is found

    for sublist in pbx_entries:
        if isinstance(sublist, list):
            for entry in sublist:
                entry_str = str(entry)
                if comp_formula and comp_formula in entry_str:
                    matches.append(entry)  # Append the entry object itself
                    found_match = True
                    break

        if found_match:
            break

    if not found_match:
        print(f"No match found for: {comp_formula}")

No match found for: Ba2 Np2 H6 O12
No match found for: Ba1 Fe2 P4 O14


In [84]:
stabilities = []

for i, x in zip(pbx_data, matches):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        stabilities.append(stability)
    except Exception as e:
        print(f"An error occurred with the element at index {pbx_data.index(i)} in pbx_data and {matches.index(x)} in matches. Error: {e}")

print(stabilities)


An error occurred with the element at index 0 in pbx_data and 0 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 1 in pbx_data and 1 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 5 in pbx_data and 5 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 6 in pbx_data and 6 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 7 in pbx_data and 7 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 8 in pbx_data and 8 in matches. Error: Composition of stability entry does not match Pourbaix Diagram
An error occurred with the element at index 9 in pbx_data and 9 in matches. Error: Composition of stability entry does not m